In [1]:
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
import numpy as np
import requests
display = InteractiveShell.ast_node_interactivity
display = "last" # "all"

In [2]:
request = requests.get("https://www.einstein.yu.edu/education/md-program/admissions/match-results-2017.aspx")

In [3]:
raw_table = BeautifulSoup(request.content, 'lxml')
raw_table = raw_table.find_all('table')[0]
raw_table = pd.read_html(str(raw_table))[0]

pd.set_option('max_rows', 8)
raw_table

,Anesthesiology,Anesthesiology.1
0,Brigham & Womens Hosp-MA,NYU School Of Medicine
1,Montefiore Med Ctr/Einstein-NY,NYU School Of Medicine
2,NYP Hosp-Columbia Univ Med Ctr-NY,UC San Francisco-CA
3,NYP Hosp-Columbia Univ Med Ctr-NY,Vanderbilt Univ Med Ctr-TN
...,...,...
127,NYP Hosp-Weill Cornell Med Ctr-NY,NaN
128,NaN,NaN
129,Urology,Urology
130,NYU School Of Medicine,Stony Brook Teach Hosps-NY


In [4]:
table = np.array(raw_table)

## Insert missing 0th entry; automatically flattens (reshape 264 1).
## Insert missing 1st entry; already flattened.
table = np.insert(table,0,'Anesthesiology')
table = np.insert(table,0,'Anesthesiology')
table = pd.DataFrame(table)

## Split table at each NaN
tables = np.split(table, np.where([pd.isnull(table[0])])[True])

## Remove NaN entries
tables = [table[~pd.isnull(table[0])] for table in tables if not isinstance(table, np.ndarray)]

## Remove empty DataFrames
tables = [table for table in tables if not table.empty]

## Read 'Specialization' from rows 0 (and 1)
for table in tables:
    table['Specialization'] = table[0].iloc[0]
tables = [table[2:] for table in tables]

result = pd.concat(tables)
result = result.rename({0: 'Institution'}, axis='columns')
result = result.reindex(columns=['Specialization','Institution'])

def state(row):
    result = row['Institution'][-2:]
    if result.isupper():
        return result
    else:
        return np.NaN

result['State'] = result.apply(state, axis=1)
result = result.sort_values(by=['State', 'Institution', 'Specialization'])
result = result.reset_index(drop=True)

pd.set_option('max_rows', 200)
result

,Specialization,Institution,State
0,Pediatrics,Childrens Hospital-Oakland-CA,CA
1,Pediatrics,Childrens Hospital-Oakland-CA,CA
2,Obstetrics-Gynecology,Kaiser Permanente-Oakland-CA,CA
3,Emergency Medicine,Loma Linda University-CA,CA
4,Pediatrics,Stanford Univ Progs-CA,CA
5,Emergency Medicine,UC Davis Med Ctr-CA,CA
6,Internal Medicine,UC San Diego Med Ctr-CA,CA
7,Anesthesiology,UC San Francisco-CA,CA
8,Pediatrics,UC San Francisco-CA,CA
9,Internal Medicine,UCLA Medical Center-CA,CA
